In [10]:
import pandas as pd
import plotly.express as px
import json
import numpy as np
from collections import Counter
pd.options.mode.copy_on_write = True

## Channel Distribution (sub v.s video)

In [28]:
# This section of for pulling data directly from db
# with open('config/secrets.json') as f:
#     config = json.load(f)
# db = config['postgres']
# con = f"postgresql://{db['user']}:{db['password']}@{db['host']}/{db['dbname']}"
# pd.options.mode.copy_on_write = True
# channel = pd.read_sql('select c.name,c.topic, cl.* from channel_log cl  left join channel c  using(channel_id)',con)

# This section of for pulling data from sample_data
c = pd.read_csv('../sample_data/channel.csv')
cl = pd.read_csv('../sample_data/channel_log.csv')
channel = pd.merge(c[['name','topic','channel_id']],cl,how='right',on='channel_id')
channel.fillna('NULL',inplace=True)
del c,cl

In [20]:
channel = channel.drop_duplicates(subset=['name'],keep='last')
topic_list = channel.topic.to_list()
topic_list = [item.replace(' ','').split(',') for item in topic_list]
topic_list = [item for row in topic_list for item in row]

In [25]:
c = Counter(topic_list)
top_topics = [item[0] for item in c.most_common()[:7]]
print(top_topics)
for idx,item in enumerate(top_topics):
    if item == 'Lifestyle_(sociology)':
        top_topics[idx] = 'Lifestyle'
    elif 'Music' in item:
        top_topics[idx] = 'Music'
print(top_topics)

['Lifestyle_(sociology)', 'Entertainment', 'Food', 'Film', 'Music', 'Music_of_Asia', 'Society']
['Lifestyle', 'Entertainment', 'Food', 'Film', 'Music', 'Music', 'Society']


In [21]:
def check_category(x):
    for topic in top_topics:
        if topic in x:
            return topic
    return 'Other'
channel['topic'] = channel['topic'].apply(check_category)

In [26]:
fig = px.scatter(channel, x="video_count", y="sub_count",color='topic',
                 log_x=True,
                 size='view_count',
                 labels={
                     "sub_count": "Num of Subscriber",
                     "video_count": "Num of video (log)",
                     "name": "Channel Name",
                     "topic": "Categories",
                     "view_count":"Num of views"
                 },
                 hover_name='name',
                 hover_data={
                    "topic":True,
                    "sub_count":True,
                    "video_count":True,
                    "video_count":False,
                },
                 category_orders={
                              "topic": [*top_topics,"Other"]},
                 title="Top youtube channels in TW",
                #  autosize=False,
                 width=800,
                 height=500,                
                 template="plotly_dark"
                 )
fig.show()

/Users/Harry/Documents/GitHub/top_youtuber_dashboard_tw/.venv/lib/python3.11/site-packages/plotly/express/_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



In [27]:
fig.write_image('../images/channel_distribution.png')
# fig.write_html('../images/channel_distribution.html')